# Walmart Retail Analytics

### I. Initial Steps

Import the packages

In [1]:
# libraries
import pandas as pd
import numpy as np
import seaborn as sns

# built in packages
import math
from datetime import datetime

# hide warnings
import warnings
warnings.filterwarnings('ignore')

Load the queried data <br>
`Queried using Big Query: https://console.cloud.google.com/bigquery?sq=936168413614:1d9ff9c5249a4e1eae5bc257061cb810`

In [2]:
data = pd.read_csv('walmart train set with features.csv')

Take a peak

In [3]:
data.head()

,Store,Date,Weekly_Sales,IsHoliday,Dept,CPI,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,Unemployment
0,1,2010-08-27,15793.87,False,1,211.567306,85.22,2.619,NaN,NaN,NaN,NaN,NaN,7.787
1,1,2010-08-27,49623.53,False,2,211.567306,85.22,2.619,NaN,NaN,NaN,NaN,NaN,7.787
2,1,2010-08-27,51159.17,False,3,211.567306,85.22,2.619,NaN,NaN,NaN,NaN,NaN,7.787
3,1,2010-08-27,36404.60,False,4,211.567306,85.22,2.619,NaN,NaN,NaN,NaN,NaN,7.787
4,1,2010-08-27,13570.32,False,5,211.567306,85.22,2.619,NaN,NaN,NaN,NaN,NaN,7.787


### II. Data Preprocessing for Visualization

Extract the year, month, and day from the Date column

In [4]:
data['Month'] = pd.DatetimeIndex(data['Date']).month
data['Day'] = pd.DatetimeIndex(data['Date']).day
data['Year'] = pd.DatetimeIndex(data['Date']).year

Replace the numeric value of the month column with the equivalent Month

In [5]:
Months = {1: 'J', 2: 'F', 3: 'M', 4: 'A', 5: 'My', 6: 'Jn', 7: 'Jl',
         8: 'Au', 9: 'S', 10: 'O', 11: 'N', 12: 'D'}

data['Month'] = data['Month'].apply(lambda x: Months[int(x)])

Get rid of the Date column

In [6]:
data.drop(['Date'], axis=1)

,Store,Weekly_Sales,IsHoliday,Dept,CPI,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,Unemployment,Month,Day,Year
0,1,15793.87,False,1,211.567306,85.22,2.619,NaN,NaN,NaN,NaN,NaN,7.787,Au,27,2010
1,1,49623.53,False,2,211.567306,85.22,2.619,NaN,NaN,NaN,NaN,NaN,7.787,Au,27,2010
2,1,51159.17,False,3,211.567306,85.22,2.619,NaN,NaN,NaN,NaN,NaN,7.787,Au,27,2010
3,1,36404.60,False,4,211.567306,85.22,2.619,NaN,NaN,NaN,NaN,NaN,7.787,Au,27,2010
4,1,13570.32,False,5,211.567306,85.22,2.619,NaN,NaN,NaN,NaN,NaN,7.787,Au,27,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421565,45,1776.44,False,93,189.421473,31.85,3.533,3205.9,4038.51,2.19,488.63,2344.09,8.424,J,20,2012
421566,45,4120.46,False,94,189.421473,31.85,3.533,3205.9,4038.51,2.19,488.63,2344.09,8.424,J,20,2012
421567,45,50973.62,False,95,189.421473,31.85,3.533,3205.9,4038.51,2.19,488.63,2344.09,8.424,J,20,2012
421568,45,6371.68,False,97,189.421473,31.85,3.533,3205.9,4038.51,2.19,488.63,2344.09,8.424,J,20,2012


Looking at the measures of the data, we will notice that the minimum value of Weekly Sales is negative. This is problematic because we're talking about Sales, not revenue so it can't be negative. Since we don't have info on whether a robbery happened in this stores or if it was a typographical error, we have to remove the negative values.

In [7]:
data.describe()

,Store,Weekly_Sales,Dept,CPI,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,Unemployment,Day,Year
count,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,150681.000000,111248.000000,137091.000000,134967.000000,151432.000000,421570.000000,421570.000000,421570.000000
mean,22.200546,15981.258123,44.260317,171.201947,60.090059,3.361027,7246.420196,3334.628621,1439.421384,3383.168256,4628.975079,7.960289,15.673131,2010.968591
std,12.785297,22711.183519,30.492054,39.159276,18.447931,0.458515,8291.221345,9475.357325,9623.078290,6292.384031,5962.887455,1.863296,8.753549,0.796876
min,1.000000,-4988.940000,1.000000,126.064000,-2.060000,2.472000,0.270000,-265.760000,-29.100000,0.220000,135.160000,3.879000,1.000000,2010.000000
25%,11.000000,2079.650000,18.000000,132.022667,46.680000,2.933000,2240.270000,41.600000,5.080000,504.220000,1878.440000,6.891000,8.000000,2010.000000
50%,22.000000,7612.030000,37.000000,182.318780,62.090000,3.452000,5347.450000,192.000000,24.600000,1481.310000,3359.450000,7.866000,16.000000,2011.000000
75%,33.000000,20205.852500,74.000000,212.416993,74.280000,3.738000,9210.900000,1926.940000,103.990000,3595.040000,5563.800000,8.572000,23.000000,2012.000000
max,45.000000,693099.360000,99.000000,227.232807,100.140000,4.468000,88646.760000,104519.540000,141630.610000,67474.850000,108519.280000,14.313000,31.000000,2012.000000


After counting the negative values in the dataset, there are 1285 negative values. We can remove it since overall we have more than a hundred thousand data.

In [8]:
data['Weekly_Sales'].value_counts(bins=[-100000, -0.00000000001, 1000000])

(-1e-11, 1000000.0]      420285
(-100000.001, -1e-11]      1285
Name: Weekly_Sales, dtype: int64

To easily remove the negative values, convert them into nan.

In [9]:
data['Weekly_Sales'] = data['Weekly_Sales'].apply(lambda x: float('nan') if x < 0 else x)

This verifies that we have 1285 null values for the Weekly_Sales column. We will drop these null values later.

In [10]:
data['Weekly_Sales'].isnull().sum()

1285

### III. Separate the dataset into two:

***Dataset 1:* To get rid of the null values and be able to analyze the effect of different variables on weekly sales, drop all the Markdown columns.**

This is because there are a lot of null values in these columns.

In [11]:
MarkDown_array = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']

data_no_mark_down = data.drop(MarkDown_array, axis=1)

Drop the rows with null values for the new dataframe.

In [12]:
data_no_mark_down.dropna(inplace=True)

Save the preprocessed data into a csv file.

In [13]:
data_no_mark_down.to_csv('data_no_mark_down.csv')

***Dataset 2:* To get the effect of Markdowns on Weekly Sales, drop the rows that contain no information on any Markdown.**

The Markdown values was converted into absolute value to get rid of the negative since it doesn't make sense that Walmart will record their Markup values. It was also converted into string so that the nan values will turn into a string 'nan.'

In [14]:
for i in MarkDown_array:
    data[i] = data[i].apply(lambda x: str(abs(x)))

A new column 'MD' was temporarily created to store the concatenated values of the 5 MarkDown columns. We turned nan into a string so those rows with null values for all the MarkDown columns will have 'nannannannannan' for their 'MD' column. That way, we can distinguish them and remove them.

In [15]:
data['MD'] = data['MarkDown1'] + data['MarkDown2'] + data['MarkDown3'] + data['MarkDown4'] + data['MarkDown5']

To be able to easily remove those rows, they were turned into nan, which is a None type.

In [16]:
data['MD'] = data['MD'].apply(lambda x: float('nan') if x == 'nannannannannan' else x)

A single line of code was run to remove all the rows with null values. The next line removed the column 'MD' since it's not needed anymore.

In [17]:
data.dropna(inplace=True)
data = data.drop(['MD'], axis = 1)

The preprocessed data was saved into a csv file.

In [18]:
data.to_csv('data_with_markdown.csv')

*NEXT: VISUALIZATION*

### IV. Data Preprocessing for Machine Learning

Load the unknown dataset.

In [19]:
Unknown_set = pd.read_csv('test.csv')
Unknown_set.head()

,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False


Extract Month, Year, and Day from Date and then drop it.

In [20]:
Unknown_set['Month'] = pd.DatetimeIndex(Unknown_set['Date']).month
Unknown_set['Year'] = pd.DatetimeIndex(Unknown_set['Date']).year
Unknown_set['Day'] = pd.DatetimeIndex(Unknown_set['Date']).day
Unknown_set = Unknown_set.drop("Date", axis = 1)

Drop Date from the train set as well. (The train set used is the one with markdown.)

In [21]:
data = data.drop('Date', axis=1)

Get rid of missing values in the MarkDown columns by converting them into zero.

In [22]:
data['MarkDown1'] = data['MarkDown1'].apply(lambda x: 0 if x == 'nan' else x)
data['MarkDown2'] = data['MarkDown2'].apply(lambda x: 0 if x == 'nan' else x)
data['MarkDown3'] = data['MarkDown3'].apply(lambda x: 0 if x == 'nan' else x)
data['MarkDown4'] = data['MarkDown4'].apply(lambda x: 0 if x == 'nan' else x)
data['MarkDown5'] = data['MarkDown5'].apply(lambda x: 0 if x == 'nan' else x)

Numerically encode Month, IsHoliday, and Year.

In [23]:
Months = {'J': 1, 'F': 2, 'M': 3, 'A': 4, 'My': 5, 'Jn': 6, 'Jl': 7,
         'Au': 8, 'S': 9, 'O': 10, 'N': 11, 'D': 12}

data['Month'] = data['Month'].apply(lambda x: Months[x])

Numerically encode IsHoliday and Year. For Year, instead of encoding, subtract the year by 2010 to get a range from 1-3. This is because the year given in the unknown dataset is different with that of the train set.

In [24]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

data['IsHoliday'] = le.fit_transform(data['IsHoliday'])
Unknown_set['IsHoliday'] = le.transform(Unknown_set['IsHoliday'])
print(le.classes_)
    
data['Year'] = data['Year'].apply(lambda x: x-2010)
Unknown_set['Year'] = Unknown_set['Year'].apply(lambda x: x-2010)

[False  True]


**Missing features in the test set:** Fuel_Price, Unemployment, CPI, Temperature

In [25]:
Unknown_set.head()

,Store,Dept,IsHoliday,Month,Year,Day
0,1,1,0,11,2,2
1,1,1,0,11,2,9
2,1,1,0,11,2,16
3,1,1,1,11,2,23
4,1,1,0,11,2,30


Below shows that Store and CPI is 1:1, which means that each store has the same CPI per week. Therefore the important variables for CPI are Store, IsHoliday, Month, and Day (Week number).

In [26]:
data.query('Month == 1 & Day == 6')[['Store','CPI']].value_counts().sort_index()

Store  CPI       
1      219.714258    72
2      219.355063    72
3      223.192305    64
4      130.157516    72
5      220.296920    63
6      221.308802    71
7      196.545831    69
8      223.240967    71
9      223.433241    64
10     130.157516    71
11     223.192305    71
12     130.157516    68
13     130.157516    74
14     189.194056    69
15     136.698129    69
16     196.545831    66
17     130.157516    71
18     136.698129    70
19     136.698129    71
20     212.571112    72
21     219.355063    67
22     140.646136    68
23     136.698129    71
24     136.698129    72
25     212.571112    68
26     136.698129    69
27     140.646136    71
28     130.157516    69
29     136.698129    68
30     219.355063    50
31     219.355063    70
32     196.545831    69
33     130.157516    45
34     130.157516    70
35     140.646136    67
36     218.406288    47
37     218.406288    53
38     130.157516    53
39     218.406288    69
40     136.698129    70
41     196.545831    7

We can carry over this to the test set, however, below shows that the "Day" feature would be different. With that, we need to replace the Day variable with Week 1-4.

In [27]:
print(data.query('Month == 1')['Day'].unique().tolist())
print(Unknown_set.query('Month == 1')['Day'].unique().tolist())

[27, 6, 13, 20]
[4, 11, 18, 25]


*This function replaces Day to 0-4 depending on what week number it is in its month. Final_df is now the new DataFrame.*

In [28]:
def Week_num(arr, df):
    final_df = pd.DataFrame()
    i = 0
    while i < len(arr):
        query_df = df.query(arr[i])
        reference_list = query_df['Day'].unique().tolist()
        reference_list.sort()
        query_df['Week'] = query_df['Day'].apply(lambda x: reference_list.index(x))
        final_df = pd.concat([query_df, final_df], ignore_index=True)
        i += 1
    return final_df
        
    
expr_array = ['Month==1', 'Month==2', 'Month==3', 'Month==4', 'Month==5', 'Month==6', 'Month==7', 'Month==8', 
             'Month==9', 'Month==10', 'Month==11', 'Month==12']

Train_set = Week_num(expr_array, data)
Unknown_set = Week_num(expr_array, Unknown_set)

Below shows that the feature 'Day' now only contains numbers 0-4 which signifies its week number.

In [29]:
Train_set['Week'].value_counts().sort_index()

0    35547
1    35561
2    35558
3    32467
4    11815
Name: Week, dtype: int64

In [30]:
Train_set.head()

,Store,Weekly_Sales,IsHoliday,Dept,CPI,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,Unemployment,Month,Day,Year,Week
0,1,25293.49,0,1,218.714733,48.91,3.172,5629.51,68.0,1398.11,2084.64,20475.32,7.866,12,2,1,0
1,1,45928.14,0,2,218.714733,48.91,3.172,5629.51,68.0,1398.11,2084.64,20475.32,7.866,12,2,1,0
2,1,10146.54,0,3,218.714733,48.91,3.172,5629.51,68.0,1398.11,2084.64,20475.32,7.866,12,2,1,0
3,1,38489.63,0,4,218.714733,48.91,3.172,5629.51,68.0,1398.11,2084.64,20475.32,7.866,12,2,1,0
4,1,32572.27,0,5,218.714733,48.91,3.172,5629.51,68.0,1398.11,2084.64,20475.32,7.866,12,2,1,0


In [31]:
Unknown_set.head()

,Store,Dept,IsHoliday,Month,Year,Day,Week
0,1,1,0,12,2,7,0
1,1,1,0,12,2,14,1
2,1,1,0,12,2,21,2
3,1,1,1,12,2,28,3
4,1,2,0,12,2,7,0


In [32]:
Train_set.to_csv('train_set.csv')
Unknown_set.to_csv('Unknown_set.csv')